# Demo 3: Manage and Share Cloud Raster

In [10]:
import os
import requests
import getpass
import json
import arcpy

## Create Cloud Storage Connection File

In [17]:
cloudstorename = "s3raster"
provider = "Amazon"
accesskey = getpass.getpass("accessKey: ")
secrectkey = getpass.getpass("secretKey: ")
bucket = "devsummit2020"
connectfolder = r"\\valyria\Z__DevSummit2020\Demo\ManageRaster"

arcpy.management.CreateCloudStorageConnectionFile(
    connectfolder, cloudstorename, service_provider=provider, bucket_name=bucket, access_key_id=accesskey, secret_access_key=secrectkey)

acspath = os.path.join(connectfolder, cloudstorename + ".acs")

accessKey: ········
secretKey: ········


## Create Mosaic Dataset for Cloud Raster

In [1]:
# Option 1: Create File Geodatabase
demofolder = r"\\valyria\Z__DevSummit2020\Demo\ManageRaster"
mdgdb = "mosaicdataset.gdb"

# Create mosaic dataset
demows = os.path.join(demofolder, mdgdb)
mdname = "yosemite_landsat7"
sr = "3857"
# sr = "4326" # This is the EPSG code for the projection Web Mercator

arcpy.CreateMosaicDataset_management(demows, mdname, sr)

mdpath = os.path.join(demows, mdname)

## Add Cloud Raster data to Mosaic Dataset

In [3]:
connectfolder = r"\\valyria\Z__DevSummit2020\Demo\ManageRaster"
cloudstorename = "s3raster"
acspath = os.path.join(connectfolder, cloudstorename + ".acs")

# Landsat 7 ETM Raster for 
data = os.path.join(acspath, "yosemite/Landsat7ETM")
rastertype = "Landsat 7 ETM+"
# Note: auxillary input option allows configuration of raster type settings
auxoption = "ProcessingTemplate 'Multispectral'"

# Add data to mosaic dataset, can either add single dataset or the entire folder. 
arcpy.AddRastersToMosaicDataset_management(
    mdpath, rastertype, data, aux_inputs=auxoption)

<Result '\\\\valyria\\Z__DevSummit2020\\Demo\\ManageRaster\\mosaicdataset.gdb\\yosemite_landsat7'>

## Register Cloud Store

In [7]:
# Create portal token if Server is federated with ArcGIS Enterprise
portal_sharing_url = "https://valyria.esri.com/portal/sharing/rest"

# Use server admin URL if Server is a standalone ArcGIS Server
server_url = "https://valyria.esri.com/server"
server_admin_url = "https://valyria.esri.com/server/admin"

username = input("Admin username: ")
password = getpass.getpass("Admin password: ")


# Generate publishing token (Federated Server)
gentokenurl = portal_sharing_url + "/generateToken"
params = {"f": "json", "username": username, "password": password, "expiration": "20", "client": "referer", "referer": server_url}

r = requests.post(gentokenurl, data=params, verify=False)
token = json.loads(r.text)["token"]


Admin username: admin
Admin password: ········


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'valyria.esri.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [ ]:
# Register cloud store 
server_regdatastore = server_admin_url + "/data/registerItem"
print(server_regdatastore)
print(token)

fileshare = {
  "path": "/cloudStores/s3raster",
  "type": "cloudStore",
  "provider": "amazon",
  "info": {
    "isManaged": False,
    "connectionString": "{\"credentialType\":\"accesskey\",\"accessKeyId\":\"*************\",\"secretAccessKey\":\"*******************\",\"region\":\"us-east-1\",\"defaultEndpointsProtocol\":\"https\"}",
    "objectStore": "devsummit2020"
  }
}

params = {"f": "json", "token": token, "item": json.dumps(fileshare), "referer": server_url}
r = requests.post(server_regdatastore, data=params, verify=False)

print(r.text)

In [8]:
# Define image service configuration
isdef = {
 "serviceName": "demo_yosemite_landsat7",
 "type": "ImageServer",
 "description": "",
 "capabilities": "Image, Catalog, Metadata, Mensuration",
 "provider": "ArcObjects11",
 "clusterName": "default",
 "minInstancesPerNode": 0,
 "maxInstancesPerNode": 2,
 "instancesPerContainer": 1,
 "maxWaitTime": 60,
 "maxStartupTime": 300,
 "maxIdleTime": 1800,
 "maxUsageTime": 600,
 "loadBalancing": "ROUND_ROBIN",
 "isolationLevel": "HIGH",
 "configuredState": "STARTED",
 "recycleInterval": 24,
 "recycleStartTime": "00:00",
 "keepAliveInterval": -1,
 "private": False,
 "isDefault": False,
 "maxUploadFileSize": 0,
 "allowedUploadFileTypes": "",
 "properties": {
  "path": "\\\\valyria\\Z__DevSummit2020\\Demo\\ManageRaster\\mosaicdataset.gdb\\yosemite_landsat7",
  "cacheControlMaxAge": "43200",
  "colormapToRGB": "false",
  "defaultResamplingMethod": "1",
 },
 "extensions": [],
 "frameworkProperties": {},
 "datasets": []
}

In [9]:
server_createservice = server_admin_url + "/services/createService"
print(server_createservice)
print(token)

params = {"f": "json", "token": token, "service": json.dumps(isdef), "referer": server_url}
r = requests.post(server_createservice, data=params, verify=False)

print(r.text)

https://valyria.esri.com/server/admin/services/createService
snvIuLAPq7zoP_G3_pvobPFt_5KA1LS8OGPFsXe0FIXQjnyy2-lUsw_l-256dzMUh-r4mhqnmV-xzCHl3QmDk3CJ1cTp-xxdkLMh3cUnN3zVWyWBuvorpM8Jreb2LjX3A2Mz58DiwCxUutPSk2ni8QrW5XUkpgqguotR8XqyQm0.


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'valyria.esri.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


{"status":"success"}


## Another option: Convert Raster to Cloud Raster Format with Copy Raster tool

In [ ]:
cloudraster = os.path.join(acspath, "yosemite_landsat7.crf")

arcpy.management.CopyRaster(mdpath, cloudraster)